In [1]:
!pip install mysql-connector-python


In [2]:
import mysql.connector as c
import os
from datetime import datetime, date, timedelta
import platform
import pandas as pd

In [3]:
conn=c.connect(host="localhost",user="root",passwd="Sharan@07",database="project1")
cursor=conn.cursor()




In [4]:
cursor.execute("SHOW DATABASES")



In [5]:
for i in cursor:
    print(i)

('dam',)
('dannys_diner',)
('data_anaytics',)
('digikull',)
('e_commersce',)
('events.csv',)
('fs_db2',)
('i_neuroni',)
('ineuron',)
('information_schema',)
('mysql',)
('new_sch',)
('new_schema',)
('newschema',)
('performance_schema',)
('planticx',)
('project1',)
('rooms',)
('sharan',)
('sharan_case',)
('subquery',)
('swiggy',)
('sys',)
('windowsfunction',)


In [6]:
cursor.execute("USE PROJECT1")



In [7]:

cursor.execute("SHOW TABLES")



In [8]:
for i in cursor:
    print(i)


('bills',)
('items',)
('login',)
('transaction',)


In [9]:
def clear():
    for _ in range(5):
        print()


def last_bill_no():
    cursor.execute('select max(bill_id) from bills')
    record = cursor.fetchone()
    return record



def find_item(no):
    cursor.execute('select * from items where id ={}'.format(no))
    record = cursor.fetchone()
    return record

In [10]:
def add_item():
    clear()
    print('Add New Item - Screen')
    print('-'*100)
    item_name = input('Enter new Item Name :')
    item_price = input('Enter Item Price :')
    sql = 'select * from items where item_name like "%{}%"'.format(item_name)
    cursor.execute(sql)
    record=cursor.fetchone()
    if record==None:
        sql = 'insert into items(item_name,price) values("{}",{});'.format(item_name,item_price)
        cursor.execute(sql)
        print('\n\nNew Item added successfully.....\nPress any key to continue....')
    else:
        print('\n\nItem Name already Exist.....\nPress any key to continue....')
    wait= input()


In [11]:
def modify_item():
    clear()
    print('Modify Item Details - Screen')
    print('-'*100)
    item_id = input('Enter Item ID :')
    item_name = input('Enter new Item Name :')
    item_price = input('Enter Item Price :')
    sql = 'update items set item_name = "{}", price ={} where id={}'.format(item_id,item_name,item_price)
    cursor.execute(sql)
    print('\n\nRecord Updated Successfully............')


In [12]:
def item_list():
    clear()
    sql="select * from items"
    cursor.execute(sql)
    records = cursor.fetchall()
    for row in records:
        print(row)
    print('\nPress any key to continue.....')
    wait = input()


In [13]:
def billing():
    clear()
    items =[]
    bill_no = last_bill_no()
    if bill_no[0]==None:
        bill_no =1
    else:
        bill_no = bill_no[0]+1 

    name = input('Enter customer Name :')
    phone = input('Enter Phone No :')
    today = datetime.today()
    while True:
        no = int(input('Enter item No (0 to stop) :'))
        if no <=0:
            break
        else:
            item = find_item(no)
            if item==None:
                print('Item Not found  ')
            else:
                qty = int(input('Enter Item Qty :'))
                item = list(item)
                item.append(qty)
                items.append(item)

    clear()
    print('                     A1 shopping mall             ')
    print('                     SECTOR NO 36 MG road Banglore      ')
    print('                     Phone: 6362885680, Email: A1_shoppingmall@gmail.com ')
    print('Bill No :{}        Date :{}'.format(bill_no,today))
    print('-'*100)
    print('Customer Name :{}          Phone No :{}'.format(name,phone))
    print('-'*100)
    print('Item Id     Item Name                Price            Qty         Amount ')
    print('-'*100)
    total =0
    for item in items:
        print('{:<10d} {:25s} {:.2f} {:>10d}          {:>.2f} \
            '.format(item[0],item[1],item[2],item[3],item[2]*item[3]))
        total = total +item[2]*item[3]
    print('-'*100)
    print('Total Payable amount : {}'.format(total))
    print('\nPress any key to continue........')
    #insert data into tables
    sql ='insert into bills(name,phone,bill_date) values("{}","{}","{}");'.format(name,phone,today)
    cursor.execute(sql)    
    for item in items:
        sql='insert into transaction(item_id,qty,bill_id) values({},{},{});'.format(item[0],item[3],bill_no)
        cursor.execute(sql)
    wait= input()

In [14]:
def date_wise_sell():
    clear()
    print('Sell Between Two Dates -- Screen')
    print('-'*100)
    start_date = input('Enter start Date (yyyy-mm-dd) :')
    end_date   = input('Enter End Date (yyyy-mm-dd) :')
    sql = 'select * from bills where bill_date between "{}" and "{}"'.format(start_date,end_date)
    cursor.execute(sql)
    records = cursor.fetchall()
    clear()
    print('Bill No       Customer Name         Phone No         Bill Date')
    print('-'*100)
    for row in records:
        print('{:10d} {:30s} {:20s} {}'.format(row[0],row[1],row[2],row[3]))
    print('-'*100)
    print('\n\nPress any key to continue....')
    wait= input()


In [15]:
def bill_information():
    clear()
    bill_no = input('Enter Bill Number :')
    sql = 'select b.bill_id,b.name,b.phone,b.bill_date,t.item_id,t.qty,i.item_name,i.price from bills b,transaction t,items i \
           where b.bill_id = t.bill_id AND t.item_id= i.id AND \
           b.bill_id ={};'.format(bill_no)
    cursor.execute(sql) 
    records = cursor.fetchall()
    n = cursor.rowcount
    clear()
    print("Bill No :",bill_no)
    print('-'*100)
    if n<=0:
        print('Bill number {} does not exists'.format(bill_no))
    else:
        print('Customer Name : {}  Phone No :{}'.format(records[0][1],records[0][2]))
        print('Bill Date : {}'.format(records[0][3]))
        print('-'*100)
        print('{:10s} {:30s} {:20s} {:10s} {:30s}'.format('ID','Item Name','Qty',
             'Price','Amount'))
        print('-'*100)
        for row in records:
            print('{:<10d} {:30s} {:<20d} {:.2f} {:>.2f}'.format(row[4],row[6],row[5],row[7],row[5]*row[7]))
        print('-'*100)
    print('\nPress any key to continue....')
    wait = input()   


In [16]:
def amount_collected():
    clear()
    start_date = input('Enter start Date (yyyy-mm-dd) :')
    end_date   = input('Enter End   Date (yyyy-mm-dd) :')
    clear()
    print('Amount collected between: {} and {}'.format(start_date,end_date))
    print('-'*100)
    sql = 'select sum(t.qty*i.price) from bills b,transaction t,items i \
           where b.bill_date between "{}" AND "{}" AND b.bill_id = t.bill_id AND \
           t.item_id = i.id;'.format(start_date,end_date)
    cursor.execute(sql)
    result = cursor.fetchone()
    print(result)
    print('\nPress any key to continue.....')
    wait= input()



In [17]:
def search_item():
    clear()
    item_name =input('Enter Item Name :')
    sql ='select * from items where item_name like "%{}%";'.format(item_name)
    cursor.execute(sql)
    records = cursor.fetchall()
    clear()
    print('Item Names start with :',item_name)
    print('-'*100)
    print('{:10s} {:30s} {:20s}'.format('Item ID','Item Name','Item Price'))
    print('-'*100)
    for row in records:
        print('{:<10d} {:30s} {:.2f}'.format(row[0],row[1],row[2]))
    print('-'*100)
    print('\nPress any key to continue....')
    wait= input()

In [18]:
def search_customer():
    clear()
    cust_name =input('Enter customer Name :')
    sql ='select * from bills where name like "%{}%";'.format(cust_name)
    cursor.execute(sql)
    records = cursor.fetchall()
    clear()
    print('Customer Names started with :',cust_name)
    print('-'*100)
    print('{:10s} {:30s} {:20s} {:20s}'.format('Bill No','Customer Name',
            'Phone No','Bill Date'))
    print('-'*100)
    for row in records:
        print('{:<10d} {:30s} {:20s} {:20s}'.format(row[0],row[1],
                row[2],str(row[3])))
    print('-'*100)
    print('\nPress any key to continue....')
    wait= input()

In [19]:
def search_bill():
    clear()
    bill_no = input('Enter Bill Number :')
    sql = 'select b.bill_id,b.name,b.phone,b.bill_date,t.item_id,t.qty,i.item_name,i.price from bills b,transaction t,items i \
           where b.bill_id = t.bill_id AND t.item_id= i.id AND \
           b.bill_id ={};'.format(bill_no)
    cursor.execute(sql) 
    records = cursor.fetchall()
    n = cursor.rowcount
    clear()
    print("Bill No :",bill_no)
    print('-'*100)
    if n<=0:
        print('Bill number {} does not exists'.format(bill_no))
    else:
        print('Customer Name : {}  Phone No :{}'.format(records[0][1],records[0][2]))
        print('Bill Date : {}'.format(records[0][3]))
        print('-'*100)
        print('{:10s} {:30s} {:20s} {:10s} {:30s}'.format('ID','Item Name',
              'Qty','Price','Amount'))
        print('-'*100)
        for row in records:
            print('{:<10d} {:30s} {:<20d} {:.2f} {:>.2f}'.format(row[4],row[6],row[5],row[7],row[5]*row[7]))
        print('-'*100)
    print('\nPress any key to continue....')
    wait = input()   

In [20]:
def search_menu():
    while True:
        clear()
        print('      S E A R C H    M E N U ')
        print('-'*100)
        print('1.   Item Name')
        print('2.   Customer information')
        print('3.   Bill information')
        print('4.   Back to main Menu')
        choice = int(input('\n\nEnter your choice (1..4): '))
        if choice==1:
            search_item()
        if choice==2:
            search_customer()
        if choice==3:
            search_bill()
        if choice==4:
            break

In [21]:
def report_menu():
    while True:
        clear()
        print('   R E P O R T   M E N U ')
        print('-'*100)
        print('1.   Item List')
        print('2.   Sell Between Dates')
        print('3.   Bill information')
        print('4.   Amount collected')
        print('5.   Back to main Menu')
        choice = int(input('\n\nEnter your choice (1..5): '))
        if choice==1:
            item_list()
        if choice==2:
            date_wise_sell()
        if choice==3:
            bill_information()
        if choice==4:
            amount_collected()
        if choice==5:
            break

In [22]:
def login():
    while True:
        clear()
        uname=input('enter your name:')
        upass=input('enter your password')
        cursor.execute('select *from login where name="{}"and pwd="{}"'.format(uname,upass))
        cursor.fetchall()
        rows=cursor.rowcount
        if rows!=1:
            print('invalid login details....try again')
        else:
            print('you are elegible for operating this system...')
            print('\n\n')
            print('Press any key to continue.....')
            break

In [23]:
def main_menu():
    while True:
        clear()
        print('      M A I N   M E N U')
        print('-'*100)
        print('1.   Add New Item')
        print('2.   Modify Existing Item')
        print('3.   Billing')
        print('4.   Search Menu')
        print('5.   Report Menu')
        print('6.   Exit')
        choice = int(input('\n\nEnter your choice (1..6): '))
        if choice==1:
            add_item()
        if choice==2:
            modify_item()
        if choice==3:
            billing()
        if choice==4:
            search_menu()
        if choice==5:
            report_menu()
        if choice==6:
            break



In [ ]:

if __name__=="__main__":
    clear()
    login()
    main_menu()











enter your name:sharan
enter your password1212
you are elegible for operating this system...



Press any key to continue.....





      M A I N   M E N U
----------------------------------------------------------------------------------------------------
1.   Add New Item
2.   Modify Existing Item
3.   Billing
4.   Search Menu
5.   Report Menu
6.   Exit
